In [4]:
import pandas as pd
import json
# Load the JSON event data, inserting relevant file path
with open('C:\\Users\\User\\Desktop\\Capstone Project\\sb_events.json', encoding='utf-8') as f:
    e_data = json.load(f)
# Load the JSON match data, inserting relevant file path
with open('C:\\Users\\User\\Desktop\\Capstone Project\\sb_matches.json', encoding='utf-8') as f:
    m_data = json.load(f)
# Convert the JSON data to a DataFrame
events_df = pd.json_normalize(e_data)
matches_df = pd.json_normalize(m_data)

In [5]:
events_df.columns = events_df.columns.str.replace(".name", "", regex=True)
events_df.columns = events_df.columns.str.replace("[.]", "_", regex=True)
matches_df.columns = matches_df.columns.str.replace(".name", "", regex=True)
matches_df.columns = matches_df.columns.str.replace("[.]", "_", regex=True)

In [6]:
# Check the columns in matches_df to find the match ID column
print("Matches DataFrame columns:")
print(matches_df.columns.tolist())
print(f"\nMatches DataFrame shape: {matches_df.shape}")
print("\nFirst few rows:")
print(matches_df.head())

Matches DataFrame columns:
['match_id', 'match_date', 'kick_off', 'home_score', 'away_score', 'behind_closed_doors', 'neutral_ground', 'collection_status', 'play_status', 'match_status', 'match_status_360', 'last_updated', 'last_updated_360', 'match_week', 'competition_competition_id', 'competition_country', 'competition_competition', 'season_season_id', 'season_season', 'home_team_home_team_id', 'home_team_home_team', 'home_team_home_team_gender', 'home_team_home_team_youth', 'home_team_managers', 'home_team_country_id', 'home_team_country', 'away_team_away_team_id', 'away_team_away_team', 'away_team_away_team_gender', 'away_team_away_team_youth', 'away_team_managers', 'away_team_country_id', 'away_team_country', 'metadata_data_version', 'metadata_shot_fidelity_version', 'metadata_xy_fidelity_version', 'competition_stage_id', 'competition_stage', 'stadium_id', 'stadium', 'stadium_country_id', 'stadium_country', 'referee_id', 'referee', 'referee_country_id', 'referee_country']

Matches

In [7]:
# Check events_df columns to find match ID column
print("Events DataFrame columns (first 20):")
print(events_df.columns.tolist()[:20])
print(f"\nEvents DataFrame shape: {events_df.shape}")
print("\nChecking if 'match_id' exists in events_df:")
print('match_id' in events_df.columns)

Events DataFrame columns (first 20):
['id', 'index', 'period', 'timestamp', 'minute', 'second', 'possession', 'duration', 'related_events', 'location', 'type_id', 'type', 'possession_team_id', 'possession_team', 'play_pattern_id', 'play_pattern', 'team_id', 'team', 'tactics_formation', 'tactics_lineup']

Events DataFrame shape: (1244341, 162)

Checking if 'match_id' exists in events_df:
True


In [8]:
# Load the matches mapping file
matches_mapping = pd.read_csv('C:\\Users\\User\\Desktop\\Capstone Project\\matches_mapping.csv')
print("Matches Mapping:")
print(matches_mapping.head())
print(f"\nMapping shape: {matches_mapping.shape}")

Matches Mapping:
   statsbomb_id  wyscout_id
0       3925227     5551642
1       3925226     5551641
2       3925228     5551644
3       3925230     5551643
4       3925232     5551646

Mapping shape: (380, 2)


In [9]:
# Step 1: Add Wyscout IDs to matches_df using left join
# This keeps all matches and adds wyscout_id where available
matches_df = pd.merge(
    matches_df, 
    matches_mapping, 
    left_on='match_id', 
    right_on='statsbomb_id', 
    how='left'
)

print("Matches DataFrame after adding Wyscout IDs:")
print(f"Shape: {matches_df.shape}")
print(f"\nSample with new wyscout_id column:")
print(matches_df[['match_id', 'wyscout_id', 'match_date', 'home_team_home_team', 'away_team_away_team']].head(10))
print(f"\nNumber of matches with Wyscout ID: {matches_df['wyscout_id'].notna().sum()}")
print(f"Number of matches without Wyscout ID: {matches_df['wyscout_id'].isna().sum()}")

Matches DataFrame after adding Wyscout IDs:
Shape: (380, 48)

Sample with new wyscout_id column:
   match_id  wyscout_id  match_date  home_team_home_team  away_team_away_team
0   3925596     5552011  2024-12-08    Consadole Sapporo       Kashiwa Reysol
1   3925601     5552012  2024-12-08      Kashima Antlers    FC Machida Zelvia
2   3925597     5552013  2024-12-08           Urawa Reds      Albirex Niigata
3   3925599     5552015  2024-12-08    Kawasaki Frontale       Avispa Fukuoka
4   3925604     5552019  2024-12-08          Vissel Kobe      Shonan Bellmare
5   3925603     5552018  2024-12-08          Gamba Osaka  Sanfrecce Hiroshima
6   3925598     5552014  2024-12-08                Tokyo         Cerezo Osaka
7   3925600     5552016  2024-12-08  Yokohama F. Marinos       Nagoya Grampus
8   3925588     5552003  2024-11-30    FC Machida Zelvia       Kyoto Sanga FC
9   3925587     5552001  2024-11-30       Kashiwa Reysol          Vissel Kobe

Number of matches with Wyscout ID: 380
Numbe

In [10]:
# Step 2: Add Wyscout IDs to events_df by merging with matches
# We only need to add the wyscout_id column from matches_df
events_df = pd.merge(
    events_df,
    matches_df[['match_id', 'wyscout_id']],  # Only take match_id and wyscout_id
    on='match_id',
    how='left'
)

print("Events DataFrame after adding Wyscout IDs:")
print(f"Shape: {events_df.shape}")
print(f"\nSample events with wyscout_id:")
print(events_df[['id', 'match_id', 'wyscout_id', 'period', 'timestamp', 'type', 'team']].head(10))
print(f"\nNumber of events with Wyscout ID: {events_df['wyscout_id'].notna().sum()}")
print(f"Number of events without Wyscout ID: {events_df['wyscout_id'].isna().sum()}")

Events DataFrame after adding Wyscout IDs:
Shape: (1244341, 163)

Sample events with wyscout_id:
                                     id  match_id  wyscout_id  period  \
0  dc1f3c18-004f-45d9-8f86-71d670f9cd97   3925601     5552012       1   
1  8e81e4fd-fc8c-4efb-b9c4-6d2a55008d43   3925601     5552012       1   
2  35110677-aaec-4db6-982d-63ae9e8f5f78   3925601     5552012       1   
3  133fd42b-056e-4634-9b32-86838cab760e   3925601     5552012       1   
4  20a0dc1c-0dde-41ac-b07b-ed0265e6a540   3925601     5552012       1   
5  4ff7ee07-01fb-44c0-b669-63b31f1145ba   3925601     5552012       1   
6  164490c2-aff8-401a-b0f6-9eb07733f5ed   3925601     5552012       1   
7  1a64fb6e-ab1e-4091-b27d-12348b7b100a   3925601     5552012       1   
8  b49dcc25-b44e-4204-a69c-3aafa690999c   3925601     5552012       1   
9  3502aa30-c30d-4efb-a182-f54e2ac5a9ca   3925601     5552012       1   

      timestamp           type               team  
0  00:00:00.000    Starting XI    Kashima Antle

In [11]:
# Step 3: Filter for Goal Kick events
goal_kick_events = events_df[events_df['play_pattern'] == 'From Goal Kick'].copy()

print(f"Total events from Goal Kick sequences: {len(goal_kick_events)}")
print(f"\nGoal kick events info:")
print(f"Unique matches: {goal_kick_events['match_id'].nunique()}")
print(f"Unique Wyscout matches: {goal_kick_events['wyscout_id'].nunique()}")
print(f"Unique possession sequences: {goal_kick_events['possession'].nunique()}")

print(f"\nEvent types in goal kick sequences:")
print(goal_kick_events['type'].value_counts().head(10))

print(f"\nSample goal kick events:")
print(goal_kick_events[['id', 'match_id', 'wyscout_id', 'period', 'timestamp', 
                         'minute', 'second', 'type', 'possession', 'team']].head(15))

Total events from Goal Kick sequences: 104936

Goal kick events info:
Unique matches: 376
Unique Wyscout matches: 376
Unique possession sequences: 201

Event types in goal kick sequences:
type
Pass              32449
Ball Receipt*     29127
Carry             19422
Pressure           8865
Duel               3493
Ball Recovery      2214
Clearance          1351
Block              1231
Foul Committed      977
Foul Won            937
Name: count, dtype: int64

Sample goal kick events:
                                       id  match_id  wyscout_id  period  \
74   0e43c912-b1e0-445f-a0c4-c648f0eace10   3925601     5552012       1   
75   46b36d6e-57bd-470c-90db-c20bc240061c   3925601     5552012       1   
76   3cbfc7da-a0b0-40f1-a62b-142d3be8345e   3925601     5552012       1   
77   3e475b94-8e4c-4e2d-9a37-a616c294b7da   3925601     5552012       1   
78   103fbfba-4c3e-47b3-b892-89991ec64264   3925601     5552012       1   
79   385d0b5b-51e8-4f14-9eb9-360d652424b6   3925601     5552012  

In [12]:
# Check if 1886347 is a Wyscout ID in our data
tracking_file_id = 1886347
print(f"Checking if {tracking_file_id} exists in our data...")

# Check in matches_mapping
if tracking_file_id in matches_mapping['wyscout_id'].values:
    statsbomb_match_id = matches_mapping[matches_mapping['wyscout_id'] == tracking_file_id]['statsbomb_id'].values[0]
    print(f"✓ Found! Wyscout ID {tracking_file_id} = StatsBomb match_id {statsbomb_match_id}")
    
    # Get match details
    match_info = matches_df[matches_df['match_id'] == statsbomb_match_id]
    if len(match_info) > 0:
        print(f"\nMatch details:")
        print(f"  Date: {match_info['match_date'].values[0]}")
        print(f"  Home: {match_info['home_team_home_team'].values[0]}")
        print(f"  Away: {match_info['away_team_away_team'].values[0]}")
        print(f"  Score: {match_info['home_score'].values[0]} - {match_info['away_score'].values[0]}")
else:
    print(f"✗ Wyscout ID {tracking_file_id} not found in mapping")
    print(f"\nFirst few Wyscout IDs in our data:")
    print(matches_mapping['wyscout_id'].head(10).tolist())

Checking if 1886347 exists in our data...
✗ Wyscout ID 1886347 not found in mapping

First few Wyscout IDs in our data:
[5551642, 5551641, 5551644, 5551643, 5551646, 5551645, 5551647, 5551648, 5551649, 5551650]


In [13]:
# Load tracking data (JSONL format - one JSON object per line)
print("Loading tracking data...")
tracking_data = []
with open('C:\\Users\\User\\Desktop\\Capstone Project\\1886347_tracking_extrapolated.jsonl', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        tracking_data.append(json.loads(line))
        if i >= 4:  # Load first few lines to check structure
            break

print(f"Loaded first {len(tracking_data)} tracking frames for inspection")
print(f"\nFirst tracking frame structure:")
print(f"Keys: {tracking_data[0].keys()}")
print(f"\nFirst frame sample:")
import pprint
pprint.pprint(tracking_data[0])

Loading tracking data...
Loaded first 5 tracking frames for inspection

First tracking frame structure:
Keys: dict_keys(['frame', 'timestamp', 'period', 'ball_data', 'possession', 'image_corners_projection', 'player_data'])

First frame sample:
{'ball_data': {'is_detected': None, 'x': None, 'y': None, 'z': None},
 'frame': 0,
 'image_corners_projection': {'x_bottom_left': None,
                              'x_bottom_right': None,
                              'x_top_left': None,
                              'x_top_right': None,
                              'y_bottom_left': None,
                              'y_bottom_right': None,
                              'y_top_left': None,
                              'y_top_right': None},
 'period': None,
 'player_data': [],
 'possession': {'group': None, 'player_id': None},
 'timestamp': None}


In [14]:
# Load the Hudl Physical Data
print("Loading Hudl Physical Data...")
with open('C:\\Users\\User\\Desktop\\Capstone Project\\hudl_physical.json', encoding='utf-8') as f:
    hudl_data = json.load(f)

# Convert to DataFrame
hudl_physical = pd.json_normalize(hudl_data)

print(f"Hudl Physical Data loaded!")
print(f"Shape: {hudl_physical.shape}")
print(f"\nFirst 20 columns:")
print(hudl_physical.columns.tolist()[:20])
print(f"\nFirst few rows:")
print(hudl_physical.head())

Loading Hudl Physical Data...
Hudl Physical Data loaded!
Shape: (1061500, 10)

First 20 columns:
['matchId', 'label', 'dateutc', 'teamId', 'teamName', 'playerid', 'player', 'metric', 'phase', 'value']

First few rows:
   matchId                                  label              dateutc teamId  \
0  5551794  Yokohama F. Marinos - Sagan Tosu, 0-1  2024-07-03 10:00:00  12798   
1  5551794  Yokohama F. Marinos - Sagan Tosu, 0-1  2024-07-03 10:00:00  12798   
2  5551794  Yokohama F. Marinos - Sagan Tosu, 0-1  2024-07-03 10:00:00  12798   
3  5551794  Yokohama F. Marinos - Sagan Tosu, 0-1  2024-07-03 10:00:00  12798   
4  5551794  Yokohama F. Marinos - Sagan Tosu, 0-1  2024-07-03 10:00:00  12798   

              teamName playerid    player                     metric  \
0  Yokohama F. Marinos   299270  J. Amano  Count Medium Acceleration   
1  Yokohama F. Marinos   299270  J. Amano  Count Medium Acceleration   
2  Yokohama F. Marinos   299270  J. Amano  Count Medium Acceleration   
3  Yoko

In [15]:
# Verify that matchId in hudl_physical contains Wyscout IDs
print("Checking matchId values in Hudl Physical data:")
print(f"Unique matches in hudl_physical: {hudl_physical['matchId'].nunique()}")
print(f"Sample matchIds: {hudl_physical['matchId'].unique()[:10]}")

# Check if these IDs exist in our mapping
sample_hudl_id = hudl_physical['matchId'].iloc[0]
print(f"\nChecking if sample Hudl matchId {sample_hudl_id} exists in matches_mapping:")
print(f"Found in wyscout_id: {sample_hudl_id in matches_mapping['wyscout_id'].values}")

Checking matchId values in Hudl Physical data:
Unique matches in hudl_physical: 378
Sample matchIds: ['5551794' '5551830' '5551826' '5551827' '5551828' '5551829' '5551824'
 '5551825' '5551822' '5551823']

Checking if sample Hudl matchId 5551794 exists in matches_mapping:
Found in wyscout_id: False


In [16]:
# Check data types
print("Data types:")
print(f"hudl_physical['matchId'] type: {hudl_physical['matchId'].dtype}")
print(f"matches_mapping['wyscout_id'] type: {matches_mapping['wyscout_id'].dtype}")

# Convert hudl_physical matchId to integer for matching
hudl_physical['matchId'] = hudl_physical['matchId'].astype(int)

print(f"\nAfter conversion: {hudl_physical['matchId'].dtype}")
print(f"Checking again if {sample_hudl_id} (as int) exists in matches_mapping:")
print(f"Found: {int(sample_hudl_id) in matches_mapping['wyscout_id'].values}")

Data types:
hudl_physical['matchId'] type: object
matches_mapping['wyscout_id'] type: int64

After conversion: int32
Checking again if 5551794 (as int) exists in matches_mapping:
Found: True


In [17]:
# Step 4: Add StatsBomb IDs to hudl_physical using left join
# The Hudl Physical data uses Wyscout IDs (matchId), we add StatsBomb IDs (statsbomb_id)
hudl_physical = pd.merge(
    hudl_physical,
    matches_mapping,
    left_on='matchId',
    right_on='wyscout_id',
    how='left'
)

print("Hudl Physical Data after adding StatsBomb IDs:")
print(f"Shape: {hudl_physical.shape}")
print(f"\nColumns: {hudl_physical.columns.tolist()}")
print(f"\nNumber of records with StatsBomb ID: {hudl_physical['statsbomb_id'].notna().sum()}")
print(f"Number of records without StatsBomb ID: {hudl_physical['statsbomb_id'].isna().sum()}")
print(f"\nSample with both IDs:")
print(hudl_physical[['matchId', 'wyscout_id', 'statsbomb_id', 'label', 'player', 'metric', 'phase', 'value']].head(10))

Hudl Physical Data after adding StatsBomb IDs:
Shape: (1061500, 12)

Columns: ['matchId', 'label', 'dateutc', 'teamId', 'teamName', 'playerid', 'player', 'metric', 'phase', 'value', 'statsbomb_id', 'wyscout_id']

Number of records with StatsBomb ID: 1061500
Number of records without StatsBomb ID: 0

Sample with both IDs:
   matchId  wyscout_id  statsbomb_id                                  label  \
0  5551794     5551794       3925378  Yokohama F. Marinos - Sagan Tosu, 0-1   
1  5551794     5551794       3925378  Yokohama F. Marinos - Sagan Tosu, 0-1   
2  5551794     5551794       3925378  Yokohama F. Marinos - Sagan Tosu, 0-1   
3  5551794     5551794       3925378  Yokohama F. Marinos - Sagan Tosu, 0-1   
4  5551794     5551794       3925378  Yokohama F. Marinos - Sagan Tosu, 0-1   
5  5551794     5551794       3925378  Yokohama F. Marinos - Sagan Tosu, 0-1   
6  5551794     5551794       3925378  Yokohama F. Marinos - Sagan Tosu, 0-1   
7  5551794     5551794       3925378  Yokoham

In [18]:
# Summary of merged datasets
print("=" * 80)
print("SUMMARY: Data Merging Complete")
print("=" * 80)

print("\n1. MATCHES_DF:")
print(f"   - Total matches: {len(matches_df)}")
print(f"   - Has both StatsBomb match_id and Wyscout wyscout_id: ✓")
print(f"   - Columns: {matches_df.shape[1]}")

print("\n2. EVENTS_DF:")
print(f"   - Total events: {len(events_df):,}")
print(f"   - Has both StatsBomb match_id and Wyscout wyscout_id: ✓")
print(f"   - Goal kick events: {len(goal_kick_events):,}")
print(f"   - Columns: {events_df.shape[1]}")

print("\n3. HUDL_PHYSICAL:")
print(f"   - Total records: {len(hudl_physical):,}")
print(f"   - Has both Wyscout matchId/wyscout_id and StatsBomb statsbomb_id: ✓")
print(f"   - Unique matches: {hudl_physical['matchId'].nunique()}")
print(f"   - Unique players: {hudl_physical['playerid'].nunique()}")
print(f"   - Unique metrics: {hudl_physical['metric'].nunique()}")
print(f"   - Columns: {hudl_physical.shape[1]}")

print("\n" + "=" * 80)
print("✓ All datasets are now ready to be connected!")
print("=" * 80)
print("\nNext steps:")
print("- Use statsbomb_id to connect events_df with hudl_physical")
print("- Use wyscout_id to connect with tracking data")
print("- Aggregate events data as needed before merging with physical data")

SUMMARY: Data Merging Complete

1. MATCHES_DF:
   - Total matches: 380
   - Has both StatsBomb match_id and Wyscout wyscout_id: ✓
   - Columns: 48

2. EVENTS_DF:
   - Total events: 1,244,341
   - Has both StatsBomb match_id and Wyscout wyscout_id: ✓
   - Goal kick events: 104,936
   - Columns: 163

3. HUDL_PHYSICAL:
   - Total records: 1,061,500
   - Has both Wyscout matchId/wyscout_id and StatsBomb statsbomb_id: ✓
   - Unique matches: 378
   - Unique players: 581
   - Unique metrics: 14
   - Columns: 12

✓ All datasets are now ready to be connected!

Next steps:
- Use statsbomb_id to connect events_df with hudl_physical
- Use wyscout_id to connect with tracking data
- Aggregate events data as needed before merging with physical data
